why you picked this data where you got it( intro slide) 
look at some of images (typical for class 1 and class 2 ) 
feature extraction (not required but you can use https://scikit-learn.org/stable/modules/feature_extraction.html) 
to explicitly get features to understand what are the features making up class 1 etc 
then model fitting 
you can have a table of all models fitted with the accuracy score, precision, recall f1 
and discuss which one is winning (is type 1 worse than type 2 etc) 
then print out some examples where your model is making correct pred and incorrect pred 
then conclusions/next steps 
so in total you can have 10 slides for this
with some appendix if you want 
lastly models that we can try can be

4. keras conv2d more complex 
5 transfer learning from keras

In [16]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import cv2
from skimage.transform import resize
import glob
from sklearn.metrics import confusion_matrix
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [43]:
X = []
y = []
for directory, _, file in os.walk('data/chest_xray/train'):
    for f in file[1::4]:
        f = os.path.join(directory, f)
        img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = resize(img, (150, 150, 1))
            img = np.asarray(img)
            label=f.split('/')[-2]
            X.append(img)
            y.append(label)

In [60]:
X_test = []
y_test = []
for directory, _, file in os.walk('data/chest_xray/test'):
    for f in file[1:]:
        f = os.path.join(directory, f)
        img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = resize(img, (150, 150, 1))
            img = np.asarray(img)
            label=f.split('/')[-2]
            X_test.append(img)
            y_test.append(label)

In [61]:
X = np.asarray(X)
X_test = np.asarray(X_test)

In [62]:
X_rf = []
for i in range(len(X)):
    a = X[i].flatten()
    X_rf.append(a)
    
X_rf = np.asarray(X_rf)

X_rf_test = []
for i in range(len(X_test)):
    a = X_test[i].flatten()
    X_rf_test.append(a)
    
X_rf_test = np.asarray(X_rf_test)

In [56]:
len(y)

1305

In [57]:
rf = ensemble.RandomForestClassifier()
rf.fit(X_rf, y)
print(cross_val_score(rf, X_rf, y))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[0.91034483 0.91264368 0.93563218]


In [63]:
y_pred = rf.predict(X_rf_test)
confusion_matrix(y_test, y_pred)

array([[ 81, 152],
       [ 12, 377]])

In [66]:
datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=.2,
        height_shift_range=.2,
        rescale=1/255,
        shear_range=.2,
        zoom_range=.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255)

if K.image_data_format() == 'channels_first':
    input_shape = (3, 150, 150)
else:
    input_shape = (150, 150, 3)

In [81]:
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [82]:
train = datagen.flow_from_directory('data/chest_xray/train',
                                   target_size=(150, 150),
                                   batch_size=16,
                                   class_mode='binary')

test = test_datagen.flow_from_directory('data/chest_xray/test',
                                   target_size=(150, 150),
                                   batch_size=16,
                                   class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [83]:
model.fit_generator(train,
                   steps_per_epoch=5216 // 16,
                   epochs = 20,
                   validation_data = test,
                   validation_steps=800 // 16)
model.save_weights('weights')

Epoch 1/2
326/326 [==============================] - 104s 318ms/step - loss: 4.0974 - acc: 0.7391 - val_loss: 5.9585 - val_acc: 0.6262
Epoch 2/2
326/326 [==============================] - 101s 310ms/step - loss: 4.0975 - acc: 0.7421 - val_loss: 6.0780 - val_acc: 0.6188


In [85]:
print(model.metrics_names)
model.evaluate(test)

['loss', 'acc']
39/39 [==============================] - 5s 122ms/step


[5.978394545041597, 0.625]

In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [4]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
test_datagen = ImageDataGenerator(rescale=1/255)
train = datagen.flow_from_directory('data/chest_xray/train',
                                   target_size=(150, 150),
                                   batch_size=16,
                                   class_mode='binary')

test = test_datagen.flow_from_directory('data/chest_xray/test',
                                   target_size=(150, 150),
                                   batch_size=16,
                                   class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [7]:
model.fit_generator(train,
                   steps_per_epoch=5216 // 16,
                   epochs = 20,
                   validation_data = test,
                   validation_steps=800 // 16)
model.save_weights('weights')

Epoch 1/20
326/326 [==============================] - 151s 462ms/step - loss: 0.4081 - acc: 0.8127 - val_loss: 1.2687 - val_acc: 0.5625
Epoch 2/20
326/326 [==============================] - 147s 452ms/step - loss: 0.3903 - acc: 0.8252 - val_loss: 0.7855 - val_acc: 0.6875
Epoch 3/20
326/326 [==============================] - 147s 450ms/step - loss: 0.3822 - acc: 0.8344 - val_loss: 0.7314 - val_acc: 0.8125
Epoch 4/20
326/326 [==============================] - 141s 431ms/step - loss: 0.3328 - acc: 0.8585 - val_loss: 0.5688 - val_acc: 0.8125
Epoch 5/20
326/326 [==============================] - 135s 415ms/step - loss: 0.3437 - acc: 0.8585 - val_loss: 0.6715 - val_acc: 0.8750
Epoch 6/20
326/326 [==============================] - 124s 382ms/step - loss: 0.3264 - acc: 0.8658 - val_loss: 0.7211 - val_acc: 0.7500
Epoch 7/20
326/326 [==============================] - 121s 372ms/step - loss: 0.3283 - acc: 0.8633 - val_loss: 0.5480 - val_acc: 0.7500
Epoch 8/20
326/326 [============================